In [1]:
!pip install --quiet google-cloud-bigquery

# Import libraries
from google.cloud import bigquery
from google.colab import auth

# Authenticate your Google account
auth.authenticate_user()

# Set up BigQuery client (use your GCP project ID)
project_id = 'akashapiplayground'  # Replace with your actual project ID
client = bigquery.Client(project=project_id)

In [2]:
# ICD9. 430. Subarachnoid hemorrage
# Filter those diagnoses where billing was for 430
query = """

  SELECT *
  FROM physionet-data.mimiciii_clinical.diagnoses_icd
  WHERE icd9_code  IN ('430')
  """

query_job = client.query(query)
diagnoses = query_job.to_dataframe()
arr_subject_id=[]
arr_hadm_id=[]
for row in range(0, len(diagnoses)):
  if(diagnoses.loc[row, 'ICD9_CODE'] in ['430']):
    arr_subject_id.append(diagnoses.loc[row, 'SUBJECT_ID'])
    arr_hadm_id.append(diagnoses.loc[row, 'HADM_ID'])

print(f"{len(arr_subject_id)=}")

len(arr_subject_id)=658


In [3]:
# Filter those notevents where subejct_id and hadm_id matches the one returned
# from diagnoses_icd query
query = """

  SELECT *
  FROM physionet-data.mimiciii_notes.noteevents
  """

query_job = client.query(query)
noteevents = query_job.to_dataframe()
df_noteevents_filtered = noteevents[noteevents['SUBJECT_ID'].isin(arr_subject_id) & noteevents['HADM_ID'].isin(arr_hadm_id)]
df_discharge_notes = df_noteevents_filtered[df_noteevents_filtered['CATEGORY'] == 'Discharge summary']
print(f"{len(df_discharge_notes)=}")
df_discharge_notes.to_csv('df_discharge_notes_430.csv', index=False)

len(df_discharge_notes)=697
